## 文档的创建和实例

In [2]:
from mongoengine import connect, disconnect
from mongoengine import Document
from mongoengine import StringField, IntField
from mongo_config import TEST_DB1, TEST_DB2, TEST_DB3, HOST, PORT, USERNAME, PASSWORD

In [4]:
from mongoengine import BooleanField, DateTimeField, DictField, EmailField, FloatField, ListField, ReferenceField

In [3]:
# 连接到已创建的数据库
connect(TEST_DB1, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB1, alias=TEST_DB1)
connect(TEST_DB2, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB2, alias=TEST_DB2)
connect(TEST_DB3, host=HOST, port=PORT, username=USERNAME, password=PASSWORD, authentication_source=TEST_DB3, alias=TEST_DB3)

MongoClient(host=['192.168.2.172:27017'], document_class=dict, tz_aware=False, connect=True, read_preference=Primary(), uuidrepresentation=3)

### 嵌入式文档，字段 EmbeddedDocuments，EmbeddedDocumentField, 字典字段DictFiled 

MongoDb能够将文档嵌入到其他文档中。可以为这些嵌入式文档定义模式，就像常规文档定义一样，但继承自EmbeddedDocuments而不是Documents。  
嵌入文档(可以是正常文档)，是以 EmbeddedDocumentField 的形式包含被嵌入文档的。
当然，从储存数据结果角度， 字典字段DictField也可以储存相同的内容，但失去了 数据模式检查字段规范的能力。
单独的嵌入文档 类，并不单独储存值集合中，也是区别 引用字段ReferenceField的方面。

In [8]:
from mongoengine import EmbeddedDocument, EmbeddedDocumentField
class Author(EmbeddedDocument):
    name = StringField()
    age = StringField()

In [9]:
class BookWithAuthor(Document):
    bookid = StringField()
    name = StringField()
    chapter = StringField(required=True, default="2.3-2.4")
    author = EmbeddedDocumentField(Author)
    meta = {"db_alias": TEST_DB1}

In [10]:
author = Author(name="司马迁", age="西汉")
book_a = BookWithAuthor(bookid="wx93ll700", name="史记", author=author)
book_a.save()

<BookWithAuthor: BookWithAuthor object>

mongodb保存结果:
```python
{
    "_id" : ObjectId("63fe09b96afa912dc75ba51e"),
    "bookid" : "wx93ll700",
    "name" : "史记",
    "chapter" : "2.3-2.4",
    "author" : {
        "name" : "司马迁",
        "age" : "西汉"
    }
}
```

In [11]:
# 使用DictFiled 保存
class BookDict(Document):
    bookid = StringField()
    name = StringField()
    chapter = StringField(required=True, default="2.3-2.4")
    author = DictField()
    meta = {"db_alias": TEST_DB1}

In [12]:
book_d = BookDict(bookid="wx93l56700", name="史记", author={'name':"司马迁", 'age':"西汉"})
book_d.save()

<BookDict: BookDict object>

mongodb保存结果:
```python
{
    "_id" : ObjectId("63fe0ab16afa912dc75ba51f"),
    "bookid" : "wx93l56700",
    "name" : "史记",
    "chapter" : "2.3-2.4",
    "author" : {
        "name" : "司马迁",
        "age" : "西汉"
    }
}
```

#### 小结：嵌入式文档还是字典字段？
从结果看，字典字段和嵌入式文档保存结果一致；但中间的检查过程是使用嵌入式文档的优势。

通常，可以使用嵌入式文档代替字典——通常建议使用嵌入式文档，因为字典不支持验证或自定义字段类型。然而，有时您并不知道要存储的内容的结构；在这种情况下使用DictFile是合适的.
字典可以存储复杂的数据、其他字典、列表、对其他对象的引用，因此是最灵活的可用字段类型。
